# Entrega 1: Visualización conjunto de datos

 Contexto: deseamos predecir cuánto tiempo estarán en tierra aquellos aviones en puntos de espera, es
 decir, cuánto tiempo transcurre entre el punto de espera y el despegue. Por ahora, en lugar de comprobar
 puntos de espera consideraremos tiempo desde la última vez que está parado (obviamente, en tierra) hasta
 que está en el aire.

El conjunto de esta entrega corresponde a los registros de los vuelos de la primera semana.
Que se encuentran en un csv y están codificados.

Entonces, lo primero que haremos será leer el csv y guardar los datos en la carpeeta datos

In [2]:
from preprocess.reader import read_data

TAR_PATH = "C:/Users/bryan/PycharmProjects/despegues/src/202412010000_202412072359.tar"
FILE_NAME = "202412010000_202412072359.csv"
read_data(TAR_PATH, FILE_NAME) # Se aplica paralelismo, sim embargo, sigue tardando un par de horas

        ts_kafka               message
0  1733011203260  jUoZIupCiFgBPAjykJ4=
1  1733011203260  oAAUllilYmBIYPDDtec=
2  1733011203260  oAAZEMMJ5TEgP//kyUg=
3  1733011203260  jQIBK/gjAAIASbh2gt8=
4  1733011203260          XTRCE1QV9w==


Ahora los datos ya se encuentran procesados y en parquets (para observar como funciona por dentro recomendamos ver)

Leeremos los datos parquets y los almazenamos en dataframes

In [ ]:
# Codigo para leer todo el conjunto y hacer concat
datos = ...
datos

In [ ]:
# Codigo para separar por semanas

## Ejercicio 1 Con los datos proporcionados, genera visualizaciones que permitan entender el conjunto de datos. 

En particular,

 (a) tráfico aéreo por horas, distinguiendo aviones en tierra y en el aire;

 (b) distribución de los tiempos de espera

 Histograma para ver cómo se distribuyen los tiempos de espera en general.
 Boxplot para identificar la mediana, los cuartiles y posibles valores atípicos.
 Mapa de calor para ver qué horas del día tienen mayor tiempo de espera.

 (c) (opcional) completa con otras visualizaciones que consideres interesantes para el problema pro
puesto;

 (d) (opcional) filtra, según la gráfica, por días, por franjas horarias, por vuelos, por tipo de avión,
 por pistas, etc.

In [ ]:
from visualization import dash
from visualization import maps

# codido para visualizar

 ## Ejercicio 2 Con los datos proporcionados, visualiza la información en un mapa.
 
 En concreto,

 (a) sitúa las pistas;

 (b) sitúa los aviones;

 (c) indica si están en tierra o en el aire;

 (d) (opcional) indica dirección;

 (e) (opcional) indica velocidad;

 (f) (opcional) indica trayectorias por avión y vuelo


In [ ]:
from preprocess.dataframe_processor import DataframeProcessor
from visualization.maps import Maps

Para visualizar la información en mapas, lo primero que haremos es eliminar de los datos aquellos aviones que consideramos "ouliers" siguiendo el siguiente criterio:

- `Un vuelo es outlier si recorre una distancia mayor a 200km en menos de 1 minuto.`

In [ ]:
df = DataframeProcessor.removeOutlierFlights(datos)

### 1. Mapa de dispersión de los estados de vuelo

En este mapa se posicionan todos los mensajes detectados en un periodo de tiempo determinado, permitiendo distinguir si la aeronave está __en tierra (on-ground)__ o __en el aire (airborne)__. Haciendo hover sobre los puntos se puede observar información adicional (ICAO, Callsign, etc.). Asimismo, se posicionan el __radar__ y las cuatro __pistas__ del aeropuerto. Por lo tanto, se trata de una primera aproximación para observar el tráfico aéreo.

In [ ]:
# PARA ESTE MAPA JAVI REDUJO EL NÚMERO DE PUNTOS QUE SE MUESTRAN PARA QUE NO HAYA DEMASIADOS PERO NO TENGO EL CÓDIGO
df_pos = DataframeProcessor.getPositions(df)
m = Maps.positionsScatterMap(df_pos)
m

In [ ]:
# PARA GUARDAR ESTOS MAPAS COMO HTML
m.save("nombre.html")

### 2. Mapa de calor

Esta visualización deriva de la anterior, se trata de un mapa de calor animado que permite monitorizar la densidad de mensajes en ventanas de tiempo muy reducidas (hemos fijado los frames a cinco minutos). De este modo, podemos observar el tráfico aéreo de forma mucho más precisa y en contexto. Se incluyen tanto las señales emitidas por aeronaves situadas tanto en tierra como en el aire.

In [ ]:
df_pos = DataframeProcessor.getPositions(df)
m = Maps.positionsHeatMap(df_pos)
m

### 3. Trayectorias por tipo de avión

El siguiente mapa permite visualizar las __trayectorias__ de las aeronaves por ICAO e identificador de vuelo distinguiendo por categoría de turbulencia. Asimismo, permite la opción de filtrar las trayectorias por despegues, aterrizajes y otros vuelos ajenos al aeropuerto. Es preciso indicar que las trayectorias se han generalizado a rectas con la finalidad de poder abarcar ventanas temporales más grandes. El punto en el final de uno de los extremos indica la __dirección__ del vuelo.

In [ ]:
df_flights = DataframeProcessor.getFlightsInfo(df)
m = Maps.trajectoriesMap(df_flights)
m

### 4. Trayectorias con velocidad

POR COMPLETAR

### 5. Trayectorias con altitud

POR COMPLETAR

In [ ]:
df_alt = DataframeProcessor.getAltitudes(df)
m = Maps.altitudesMap(df_alt)
m